In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
# este código realiza una salida de una tabla con datos ya previamente procesados, quedándonos las partes que nos interesan. 

spam_or_ham = pd.read_csv("dataset//spam.csv", encoding='latin-1')[["v1", "v2"]]
spam_or_ham.columns = ["label", "text"]
spam_or_ham.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Con esta línea de código se pueden contar cuántas muestras tenemos de textos clasificados como spam o no.
spam_or_ham["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [4]:
# Ahora podemos definir una función que nos servirá para realizar el proceso de tokenización y 
# eliminación de las partes innecesarias de las muestras.

import string
punctuation = set(string.punctuation)
def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [14]:
# Es un algoritmo básico que simplemente separa las frases en arrays de palabras y elimina signos de puntuación. 
# Lo normal aquí sería eliminar más elementos innecesarios pero al menos se capta la idea detrás de esta etapa 
# del proceso de machine learning. 
# Aplicamos el algoritmo de tokenización sobre la muestra con el siguiente código
spam_or_ham.head(50)["text"].apply(tokenize)

0     [go, until, jurong, point, crazy, available, o...
1                        [ok, lar, joking, wif, u, oni]
2     [free, entry, in, 2, a, wkly, comp, to, win, f...
3     [u, dun, say, so, early, hor, u, c, already, t...
4     [nah, i, dont, think, he, goes, to, usf, he, l...
5     [freemsg, hey, there, darling, its, been, 3, w...
6     [even, my, brother, is, not, like, to, speak, ...
7     [as, per, your, request, melle, melle, oru, mi...
8     [winner, as, a, valued, network, customer, you...
9     [had, your, mobile, 11, months, or, more, u, r...
10    [im, gonna, be, home, soon, and, i, dont, want...
11    [six, chances, to, win, cash, from, 100, to, 2...
12    [urgent, you, have, won, a, 1, week, free, mem...
13    [ive, been, searching, for, the, right, words,...
14           [i, have, a, date, on, sunday, with, will]
15    [xxxmobilemovieclub, to, use, your, credit, cl...
16                            [oh, kim, watching, here]
17    [eh, u, remember, how, 2, spell, his, name

In [6]:
# Ahora entramos en la parte más compleja de esta práctica, en la que usamos la librería scikit-learn a fin de realizar 
# el trabajo pesado del proceso de aprendizaje y pruebas. 
# Comenzamos con la configuración de los parámetros necesarios para  
# aprender sobre la base de las muestras obtenidas en el archivo csv

from sklearn.feature_extraction.text import CountVectorizer
demo_vectorizer = CountVectorizer(
    tokenizer = tokenize,
    binary = True
)

In [18]:
# Esta es la configuración que necesitaremos para obtener la tabla de datos ya procesados sobre las muestras, que será el
# motor que usaremos para interpretar todos los futuros mensajes y catalogarlos como spam o no. En resumen, estamos diciendo 
# qué función tiene que usar para la tokenización y que ésta debe ser binaria, es decir, no importa el número de veces que 
# aparece una palabra, simplemente mirará si aparece o no. Con el siguiente código separamos los datos entre entrenamiento 
# y pruebas.

from sklearn.model_selection import train_test_split
train_text, test_text, train_labels, test_labels = train_test_split(spam_or_ham["text"], spam_or_ham["label"], stratify=spam_or_ham["label"])
print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

train_labels.head(10)

Training examples: 4179, testing examples 1393


2024     ham
2531     ham
4309    spam
4639     ham
4634     ham
3273     ham
3328     ham
118      ham
1570     ham
4721     ham
Name: label, dtype: object

In [8]:
# Después de haber hecho esta separación podemos pasar al siguiente paso, que consiste en crear un nuevo vectorizador, 
# desde cero, en el que solamente vamos a usar los datos de entrenamiento, no los datos de pruebas.

real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
train_X = real_vectorizer.fit_transform(train_text)
test_X = real_vectorizer.transform(test_text)

In [9]:
# Ahora volvemos a usar nuevas utilidades de la librería scikit-learn para seguir con nuestro trabajo de 
# preparación del Machine Learning. En esta etapa creamos el nuevo clasificador y usamos el método fit() 
# para procesar los datos, lo que prepara al clasificador para usarlo más adelante. De nuevo, usamos los datos 
# de entrenamiento para prepararlo, no los de prueba.

from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(train_X, train_labels)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

LinearSVC()

In [10]:
"""
En este punto nuestro clasificador está listo para trabajar con él, pero antes de ello podemos realizar una operación muy 
interesante que se basa en predecir la precisión de las clasificaciones que conseguirá. Se usa el método predict() que 
permite realizar finalmente las clasificaciones.

Para calcular la precisión de nuestro sistema de Machine Learning usamos los datos de prueba, los cuales sabemos los 
resultados que deben entregar. Comparando las predicciones con los datos reales que tenían los datos de prueba podemos 
calcular la precisión que tendrá. En este bloque de código realizamos todos esos pasos para el cálculo de la precisión, 
en el que nos ayudamos de una función de scikit-learn llamada accuracy_score() que nos sirve para calcular la puntuación 
de manera sencilla
"""
from sklearn.metrics import accuracy_score
predicciones = classifier.predict(test_X)
accuracy = accuracy_score(test_labels, predicciones)
print(f"Accuracy: {accuracy:.4%}")

Accuracy: 97.9899%


In [11]:
"""
Clasificar textos nuevos y comprobar el sistema Machine Learning
Por fin podemos llegar al final de nuestra práctica, que consiste en crear una serie de frases para comprobar si son spam o no. 
Estas frases las podemos insertar en un array:
"""
frases = [
  'Are you looking to redesign your website with new modern look and feel?',
  'Please send me a confirmation of complete and permanent erasure of the personal data',
  'You have been selected to win a FREE suscription to our service',
  'We’re contacting you because the webhook endpoint associated with your account in test mode has been failing'
]

In [12]:
"""
A continuación las pasamos por nuestro algoritmo de transformación y vectorización, para finalmente recibir las 
predicciones de clasificación.
"""
frases_X = real_vectorizer.transform(frases)
predicciones = classifier.predict(frases_X)

In [13]:
# Con este último bloque podemos recorrer las predicciones y mostrar lo que nuestro sistema ha sido capaz de clasificar.

for text, label in zip(frases, predicciones):
    print(f"{label:5} - {text}")

spam  - Are you looking to redesign your website with new modern look and feel?
ham   - Please send me a confirmation of complete and permanent erasure of the personal data
spam  - You have been selected to win a FREE suscription to our service
ham   - We’re contacting you because the webhook endpoint associated with your account in test mode has been failing
